### Imports

In [1]:
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import random
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

### Download the dataset

In [ ]:
train_set = datasets.MNIST('/content',train=True,download=True)
test_set = datasets.MNIST('/content',train=False,download=True)

In [3]:
# inversion and normalization
invert = lambda image : 255 - image # function to invert the image

def preprocessing(dataset):
    dataset_images = dataset.data.numpy() # convert the dataset into numpy array
    dataset_labels = dataset.targets.numpy() # convert the labels into numpy array
    return dataset_images,dataset_labels

train_images,train_labels = preprocessing(train_set)
test_images,test_labels = preprocessing(test_set)

### Prepare the data

In [5]:
def generate_labels(dataset_images,image_type):
  labels = np.full(shape=(dataset_images.shape[0]),fill_value=[image_type])
  return labels

def binarize(dataset_images):
  result = np.copy(dataset_images)
  for i in range(result.shape[0]):
    for j in range(result.shape[1]):
      for k in range(result.shape[2]):
        if result[i][j][k] > 127:
          result[i][j][k] = 1
        else:
          result[i][j][k] = 0
  return result

# function for inverting data
def get_inverted_data(source_dataset_images,source_dataset_labels,target_dataset,class_num):
    image_indices = np.asarray(np.where(source_dataset_labels == class_num))
    image_indices = image_indices.flatten()
    original_images = invert(source_dataset_images[image_indices]) # get the original images and invert them
    target_dataset = np.concatenate((target_dataset,original_images)) # concatenate to our training dataset
    return target_dataset

# add some data from already available negative data
def add_negative_data(source_dataset_images,source_dataset_labels,target_dataset,class_num,amount=2000):
  image_indices = np.asarray(np.where(source_dataset_labels != class_num))
  image_indices = image_indices.flatten()
  np.random.shuffle(image_indices)
  image_indices = image_indices[:amount] # take very less amount of negative data
  original_images = source_dataset_images[image_indices]
  target_dataset = np.concatenate((target_dataset,original_images))
  inverted_images = invert(original_images)
  target_dataset = np.concatenate((target_dataset,inverted_images))
  return target_dataset

# function to prepare training data for a given digit
def prepare_training_data(dataset_images,dataset_labels,class_num):
    indices = np.asarray(np.where(dataset_labels == class_num)) # indices of occurrence of digit as label
    indices = indices.flatten()
    # get the images for making positive dataset
    dataset_images_positive = dataset_images[indices] # images consisting of positive class
    dataset_images_positive = binarize(dataset_images_positive) # binarize the images 
    dataset_labels_positive = generate_labels(dataset_images_positive,0) # generate the class labels
    # print(f"Positive train data shape: {dataset_images_positive.shape}")
    # get the images for making negative dataset
    dataset_images_negative = np.empty((0,28,28),dtype=np.float32)
    dataset_images_negative = get_inverted_data(dataset_images,dataset_labels,dataset_images_negative,class_num)
    # dataset_images_negative = add_negative_data(dataset_images,dataset_labels,dataset_images_negative,class_num)
    dataset_images_negative = binarize(dataset_images_negative) # binarize the images 
    dataset_labels_negative = generate_labels(dataset_images_negative,1) # generate the class labels
    # print(f"Negative train data shape: {dataset_images_negative.shape}")
    # concatenate the negative and positive datasets
    modified_dataset_images = np.concatenate((dataset_images_positive,dataset_images_negative))
    modified_dataset_labels = np.concatenate((dataset_labels_positive,dataset_labels_negative))
    return modified_dataset_images,modified_dataset_labels


def prepare_testing_data(dataset_images,dataset_labels,class_num):
  positive_indices = np.asarray(np.where(dataset_labels == class_num)) # indices of occurrence of digit as label
  positive_indices = positive_indices.flatten()
  # get the images for making positive dataset
  dataset_images_positive = dataset_images[positive_indices] # images consisting of positive class
  dataset_images_positive = binarize(dataset_images_positive)
  dataset_labels_positive = generate_labels(dataset_images_positive,0) # generate the class labels
  # print(f"Positive test data shape: {dataset_images_positive.shape}")
  # get the images for making negative classes for testing
  negative_indices = np.asarray(np.where(((dataset_labels != class_num))))
  negative_indices = negative_indices.flatten()
  # get the images for making negative dataset
  dataset_images_negative = dataset_images[negative_indices]
  dataset_images_negative = binarize(dataset_images_negative)
  dataset_labels_negative = generate_labels(dataset_images_negative,1)
  # print(f"Negative test data shape: {dataset_images_negative.shape}")
  # concatenate the negative and positive datasets
  modified_dataset_images = np.concatenate((dataset_images_positive,dataset_images_negative))
  modified_dataset_labels = np.concatenate((dataset_labels_positive,dataset_labels_negative))
  return modified_dataset_images,modified_dataset_labels

### Get training and test data

In [6]:
class_num = 0
trainX,trainy = prepare_training_data(train_images,train_labels,class_num)
testX,testy = prepare_testing_data(test_images,test_labels,class_num)

### Function for creating modified dataset

In [7]:
def modify_dataset(X,y,patterns,antipatterns):
  p = patterns.shape[0]
  ap = antipatterns.shape[0]
  modified_train_X = np.empty((0,p+ap),dtype=X.dtype)
  modified_train_y = y.copy()
  for i in range(X.shape[0]):
    image,label = X[i],y[i]
    modified_X = np.zeros(p+ap)
    for i in (range(p)):
      score = np.sum(patterns[i]*image)
      modified_X[i] = score
    for i in range(ap):
      score = np.sum(antipatterns[i]*image)
      modified_X[p+i] = score
    
    modified_train_X = np.vstack((modified_train_X,modified_X))
  
  return modified_train_X,modified_train_y

### Find patterns

In [ ]:
patterns = np.empty((0,28,28),dtype=np.float32)
antipatterns = np.empty((0,28,28),dtype=np.float32)

positive_image_indices = [i for i in range(trainX.shape[0]) if trainy[i] == 0]
positive_image = trainX[positive_image_indices[0]]
patterns = np.vstack((patterns,[positive_image]))
antipatterns = np.vstack((antipatterns,[1-positive_image]))
running_trainX,running_trainy = trainX.copy(),trainy.copy()
thresh = 0
clf_list = []
num = 0
while True:
  y_pred = []
  tempX = np.empty((0,28,28),dtype=np.float32)
  tempy = []
  modified_trainX,modified_trainy = modify_dataset(running_trainX,running_trainy,patterns,antipatterns)
  print(modified_trainX.shape)
  clf = DecisionTreeClassifier(max_depth=1)
  clf.fit(modified_trainX,modified_trainy)
  clf_list.append((str(num),clf))
  num += 1
  y_pred = clf.predict(modified_trainX)
  for i in range(modified_trainX.shape[0]):
    if y_pred[i] != modified_trainy[i]:
      tempy.append(modified_trainy[i])
      tempX = np.vstack((tempX,[running_trainX[i]]))
  print(accuracy_score(modified_trainy,y_pred))

  if accuracy_score(modified_trainy,y_pred) == 1:
    break
  tempy = np.array(tempy)
  for i in range(len(y_pred)):
    if y_pred[i] == 1 and modified_trainy[i] == 0: # found a false negative
      print("REACHED")
      patterns = np.vstack((patterns,[running_trainX[i]]))
      antipatterns = np.vstack((antipatterns,[1-running_trainX[i]]))
      break
  
  running_trainX,running_trainy = tempX.copy(),tempy.copy()

### Plot variations

In [ ]:
for i in range(patterns.shape[0]):
  if i+1 > 16:
    break
  plt.subplot(4,4,i+1)
  plt.axis("off")
  plt.imshow(patterns[i].squeeze(),cmap='gray')


In [ ]:
stacking = StackingClassifier(estimators=clf_list,stack_method='predict')
modified_testX,modified_testy = modify_dataset(testX,testy,patterns,antipatterns)
modified_trainX,modified_trainy = modify_dataset(trainX,trainy,patterns,antipatterns)
stacking.fit(modified_trainX,modified_trainy)
print(accuracy_score(modified_testy,clf.predict(modified_testX)))
print(stacking)